In [ ]:
1+1

2

# URL

- https://www.kaggle.com/competitions/playground-series-s3e19/overview

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [13]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
le = LabelEncoder()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49
5,5,2017-01-01,Argentina,Kaggle Store,Using LLMs to Improve Your Coding,88
6,6,2017-01-01,Argentina,Kaggle Store,Using LLMs to Train More LLMs,98
7,7,2017-01-01,Argentina,Kaggle Store,Using LLMs to Win Friends and Influence People,14
8,8,2017-01-01,Argentina,Kaggle Store,Using LLMs to Win More Kaggle Competitions,83
9,9,2017-01-01,Argentina,Kaggle Store,Using LLMs to Write Better,69


In [18]:
def split_date(df):
  df['date'] = pd.to_datetime(df['date'])
  df['Year'] = df['date'].dt.year
  df['Month'] = df['date'].dt.month
  df['Day'] = df['date'].dt.day
  df['WeekDay'] = df['date'].dt.dayofweek
  return df

In [15]:
def labels_encode(df):
  df['store'] = le.fit_transform(df['store'])
  df['country'] = le.fit_transform(df['country'])
  df['product'] = le.fit_transform(df['product'])
  return df

In [17]:
def train_by_lightgbm(df):
  df_train, df_val =train_test_split(df, test_size=0.2)
  train_x = df_train[['store', 'country', 'product', 'Year', 'Month', 'Day', 'WeekDay']]
  train_y = df_train[['num_sold']]

  val_x = df_val[['store', 'country', 'product', 'Year', 'Month', 'Day', 'WeekDay']]
  val_y = df_val[['num_sold']]

  trains = lgb.Dataset(train_x, train_y)
  valids = lgb.Dataset(val_x, val_y)

  params = {
      "objective": "regression",
      "metrics": "mae"
  }
  model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)
  return model



In [19]:
train = labels_encode(train)
train = split_date(train)
model = train_by_lightgbm(train)

/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 71
[LightGBM] [Info] Number of data points in the train set: 109560, number of used features: 7
[LightGBM] [Info] Start training from score 165.424936
[1]	valid_0's l1: 123.285
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 111.41
[3]	valid_0's l1: 100.743
[4]	valid_0's l1: 91.2303
[5]	valid_0's l1: 82.6708
[6]	valid_0's l1: 75.0596
[7]	valid_0's l1: 68.2467
[8]	valid_0's l1: 62.2667
[9]	valid_0's l1: 56.9681
[10]	valid_0's l1: 52.2568
[11]	valid_0's l1: 48.1218
[12]	valid_0's l1: 44.5511
[13]	valid_0's l1: 41.2462
[14]	valid_0's l1: 38.5199
[15]	valid_0's l1: 35.9636
[16]	valid_0's l1: 33.8251
[17]	valid_0's l1: 31.7434
[18]	valid_0's l1: 30.0866
[19]	valid_0's l1: 28.5305
[20]	valid_0's l1: 27.2014

In [20]:
test = labels_encode(test)
test = split_date(test)
predict = model.predict(test[['store', 'country', 'product', 'Year', 'Month', 'Day', 'WeekDay']])
test['num_sold'] = predict
test[['id', 'num_sold']].to_csv("submission.csv", index=False)